In [2]:
import json
import torch
import numpy as np
from src.backbones import utae
from datetime import datetime

In [4]:
reference_date = "2018-01-01"
reference_date_d = datetime(*map(int, reference_date.split("-")))
image_date = "2018-06-01"
image_date_d = datetime(*map(int, image_date.split("-")))

# Calculate the difference in days between the image date and the reference date
date_diff = (image_date_d - reference_date_d).days

# Load the data
data = torch.from_numpy(np.load(f"./JAXA/DATA_S2/S2_10000.npy").astype(np.float32))
data = data[0, :, :, :]
target = torch.from_numpy(np.load(f"./JAXA/ANNOTATIONS/TARGET_10000.npy").astype(np.int8))
# Load normalization values
with open("./model_deployment/model_dir/normalization.json", "r") as file:
    normvals = json.loads(file.read())
means = [normvals["Fold_{}".format(f)]["mean"] for f in range(1, 6)]
stds = [normvals["Fold_{}".format(f)]["std"] for f in range(1, 6)]
norm = np.stack(means).mean(axis=0), np.stack(stds).mean(axis=0)
norm = (
    torch.from_numpy(norm[0]).float(),
    torch.from_numpy(norm[1]).float(),
)

# Normalize the data
data = (data - norm[0][None, :, None, None]) / norm[1][None, :, None, None]
dates = torch.tensor([date_diff])
x = data.unsqueeze(dim=0), dates.unsqueeze(dim=0)

# Model configurations
device = "cpu"
model = utae.UTAE(
    input_dim=9,
    encoder_widths=[64, 64, 64, 128],
    decoder_widths=[32, 32, 64, 128],
    out_conv=[32, 4],
    str_conv_k=4,
    str_conv_s=2,
    str_conv_p=1,
    agg_mode="att_group",
    encoder_norm="group",
    n_head=16,
    d_model=256,
    d_k=4,
    encoder=False,
    return_maps=False,
    pad_value=0,
    padding_mode="reflect",
)
model = model.to(device)
sd = torch.load("artifacts/4_cl_1/fold_1/model.pth.tar", map_location=device, weights_only=True)
model.load_state_dict(sd["state_dict"])

def recursive_todevice(x, device):
    if isinstance(x, torch.Tensor):
        return x.to(device)
    elif isinstance(x, dict):
        return {k: recursive_todevice(v, device) for k, v in x.items()}
    else:
        return [recursive_todevice(c, device) for c in x]
    
x = recursive_todevice(x, device) 

with torch.inference_mode():
    out = model(x[0], batch_positions=x[1])
# pred = out.squeeze(dim=0).squeeze(dim=0).cpu().numpy()
# binary_image = np.where(pred >= 0.5, 1, 0)
pred = out.argmax(dim=1)

In [5]:
example_input = torch.randn(1, 1, 9, 128, 128).to(device)
example_dates = torch.tensor([0]).unsqueeze(dim=0).to(device)

# Trace the model
model.eval()
traced_model = torch.jit.trace(model, (example_input, example_dates))

# Save the traced model
torch.jit.save(traced_model, "model_deployment/model_dir/model_4cl.pt")

/home/thankscarbon/miras/utae/src/backbones/utae.py:195: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if pad_mask.any():
/home/thankscarbon/miras/utae/src/backbones/utae.py:361: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if pad_mask is not None and pad_mask.any():
/home/thankscarbon/miras/utae/src/backbones/utae.py:397: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might

In [1]:
import numpy as np
import json

# Load the data
data = np.load(f"./JAXA/DATA_S2/S2_10000.npy").astype(np.float32)
image_data = data[0, :, :, :]
date = "2018-06-01"

# Create the JSON structure
instance = {
    "image": image_data.tolist(),  # Convert NumPy array to a nested list
    "date": date
}

# Save to a JSON file (e.g., test.json)
with open("test.json", "w") as f:
    json.dump(instance, f, indent=4)  # Use indent for readability (optional)

print("Data saved to test.json")

Data saved to test.json
